In [ ]:
import dbnl
import json
import random
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
from datetime import UTC, datetime, timedelta

# Make sure your version matches the docs at https://docs.dbnl.com/
print("dbnl version:", dbnl.__version__)

In [ ]:
# Login to DBNL (using default Sandbox url)
dbnl.login(
    api_url="http://localhost:8080/api",
    api_token="<DBNL_API_KEY>", # found at http://localhost:8080/tokens
)

In [ ]:
# Create a new project
project = dbnl.get_or_create_project(
    name="ADK Calc Traces SDK via JSON and Augment",
    schedule="daily",  # How often DBNL analyzes new data
    default_llm_model_name="quickstart_model" # From step (2) in https://docs.dbnl.com/get-started/quickstart
)

In [ ]:
# Load the JSONL file
df = pd.read_json('traces.jsonl', lines=True)

# Convert timestamp columns to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# For nested timestamps in spans, we also need to convert to datetime
def convert_span_times(spans):
    for span in spans:
        span['start_time'] = pd.to_datetime(span['start_time'])
        span['end_time'] = pd.to_datetime(span['end_time'])
    return spans

df['spans'] = df['spans'].apply(convert_span_times)

In [ ]:
def compute_expected(cell):
    """
    Accepts either {'input': '2+2'} or a JSON string of that dict.
    Returns {'output': '<result>'} or {'output': 'cannot compute'}.
    """
    try:
        d = cell
        if isinstance(cell, str):
            d = json.loads(cell)  # tolerate JSON-encoded dicts
        if not (isinstance(d, dict) and isinstance(d.get("input"), str)):
            return json.dumps({"output": "cannot compute"})
        val = eval(d["input"])
        return json.dumps({"output": str(val)})
    except Exception:
        return json.dumps({"output": "cannot compute"})

# add the new column
df["output_expected"] = df["input"].apply(compute_expected)

In [ ]:
# Short messages (customize as you like)
complaints = [
    "That’s not right!", "Wrong result again!", "Calculator failed.",
    "Off by a mile.", "Bad math output!", "Totally incorrect!",
    "Oops, wrong calc.", "Computation error.", "Answer is wrong.",
    "Incorrect result.", "Math seems broken.", "Calculation flaw.",
    "Miscalculated that.", "Wrong total shown.", "Completely off!",
    "This seems buggy.", "Bad arithmetic!", "The math is wrong.",
    "Way off the mark.", "Error in result!"
]
praises = [
    "Perfect result!", "Nice work!", "Correct again!", "Spot on!",
    "You nailed it!", "Looks good!", "Math checks out!", "Well done!",
    "Accurate answer!", "Exactly right!", "All good here!", "Bang on target!",
    "That’s correct!", "Great calculation!", "Awesome result!",
    "Nice precision!", "Flawless math!", "Right on point!",
    "Excellent job!", "Spotless result!"
]

def parse_output_cell(cell):
    """Return a dict like {'output': <str>} or {} if unusable."""
    if isinstance(cell, dict):
        return {"output": str(cell.get("output"))} if "output" in cell else {}
    if isinstance(cell, str):
        # try JSON first
        try:
            obj = json.loads(cell)
            if isinstance(obj, dict) and "output" in obj:
                return {"output": str(obj["output"])}
        except Exception:
            # treat raw string as the value itself
            return {"output": cell}
    return {}

def compute_feedback(row, p_keep=0.11):
    # 89% chance to leave both None
    if np.random.rand() > p_keep:
        return pd.Series({"feedback_score": None, "feedback_text": None})

    out_d = parse_output_cell(row["output"])
    exp_d = parse_output_cell(row["output_expected"])

    out = out_d.get("output")
    exp = exp_d.get("output")

    if out is not None and exp is not None and out == exp:
        score = 5
        text = random.choice(praises)
    else:
        score = 1
        text = random.choice(complaints)

    # JSON-safe primitives
    return pd.Series({"feedback_score": int(score), "feedback_text": str(text)})

# Apply across the dataframe
df[["feedback_score", "feedback_text"]] = df.apply(compute_feedback, axis=1)

In [ ]:
# Adjust timestamps to current time so data appears recent and split across last 8 days
day_dfs = [chunk.reset_index(drop=True) for chunk in np.array_split(df, 8)]

for idx, df in enumerate(day_dfs):
    delta = datetime.now(tz=UTC) - df["timestamp"].max() - timedelta(days=(8-idx))
    delta = timedelta(days=round(delta / timedelta(days=1)))
    df['timestamp'] = df['timestamp'] + delta

    # For nested timestamps in spans, we also need to convert to datetime
    def convert_span_times(spans):
        for span in spans:
            span['start_time'] = span['start_time'] + delta
            span['end_time'] = span['start_time'] + delta
        return spans
    
    df['spans'] = df['spans'].apply(convert_span_times)

In [ ]:
print("Uploading data...")
print(f"See status at: http://localhost:8080/ns/{project.namespace_id}/projects/{project.id}/status")

data_start_t = df['timestamp'].min().replace(hour=0, minute=0, second=0, microsecond=0)
data_end_t = data_start_t + timedelta(days=1)

for idx, day_df in enumerate(day_dfs):
    print(f"{idx + 1} / {len(day_dfs)} publishing log data for {min(day_df['timestamp']).date()}")
    data_start_t = min(day_df['timestamp']).replace(hour=0, minute=0, second=0, microsecond=0)
    data_end_t = data_start_t + timedelta(days=1)
    try:
        dbnl.log(
            project_id=project.id,
            data_start_time=data_start_t,
            data_end_time=data_end_t,
            data=day_df,
        )
    except Exception as e:
        if "Data already exists" in str(e):
            continue
        raise

print("You can now explore your data in DBNL!")
print(f"http://localhost:8080/ns/{project.namespace_id}/projects/{project.id}")